# General note

The success and failure of the various code blocks probably depends on the access allowed by your subscription and whether you have the additional "institutional token" that must be requested specially. I don't have that, but I'm not sure which of the failures are related to that. 

It also apparently uses your IP address to determine what you can access. I'm using the Vanderbilt VPN, so presumably it should be allowing me to use whatever I can access due to the VU subscriptions.

## Run only once!

This first cell only needs to be run once to install the Elsevier elsapy SDK

In [ ]:
# If the elsapy module can't be located even though you installed it at the command line
# try this suggestion from
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
# You only need to do it once ever unless you reinstall Jupyter notebooks.

# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install elsapy

## Configuration

Import modules, define functions, set configuration variables

In [ ]:
# elsapy documentation: https://elsevierdev.github.io/elsapy/


# Import modules
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
from pathlib import Path
import time
import requests

# Configuration variables

# I am not sure what this is for. The value is the one in their example, which doesn't work
elsevier_author_id = '7004367821'

# I am not sure what this is for. The value is the one in their example, which doesn't work
elsevier_institutional_affiliation = '60101411'

# This function will load some credential from a text file, either in the home directory or current working directory
# The value of the directory variable should be either 'home' or 'working'
# Keeping the credential in the home directory prevents accidentally uploading it with the notebook.
# The function returns a single string, so if there is more than one credential (e.g. key plus secret), additional
# parsing of the return value may be required. 
def load_credential(filename, directory):
    cred = ''
    # to change the script to look for the credential in the working directory, change the value of home to empty string
    if directory == 'home':
        home = str(Path.home()) #gets path to home directory; works for both Win and Mac
        credential_path = home + '/' + filename
    else:
        directory = 'working'
        credential_path = filename
    try:
        with open(credential_path, 'rt', encoding='utf-8') as file_object:
            cred = file_object.read()
    except:
        print(filename + ' file not found - is it in your ' + directory + ' directory?')
        exit()
    return(cred)


# Attempt direct access to API without using Python client library


In [ ]:
# API specifiction landing page: https://dev.elsevier.com/api_docs.html
# For author search info, see https://dev.elsevier.com/documentation/AuthorSearchAPI.wadl

# Test the author search API
endpoint_resource_url = 'https://api.elsevier.com/content/search/author'
# NOTE: I was able to generate the API key myself using the Elsevier Developer Portal. However, for some reason it gave me access to the
# Scopus API, but not the Author Search API. I had to request access to the Author Search API from Elsevier support and they gave me
# an institutional token that is an additional requirement for me to get the Author Search API to work. 
header_parameters = {
    'Accept': 'application/json', 
    'X-ELS-APIKey': load_credential('elsevier-api-key.txt', 'home'),
    'X-ELS-Insttoken': load_credential('elsevier-inst-token.txt', 'home')
    }
query_parameters = {
    'query': 'authlast(Chen)',
    }

response = requests.get(endpoint_resource_url, headers=header_parameters, params=query_parameters)
print(response.status_code)
print(response.text)


In [ ]:
# Test the article search API
# Scopus search info: https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl
endpoint_resource_url = 'https://api.elsevier.com/content/search/scopus'
header_parameters = {
    'Accept': 'application/json', 
    'X-ELS-APIKey': load_credential('elsevier-api-key.txt', 'home')
    }
query_parameters = {
    'query': 'heart+attack',
    }

response = requests.get(endpoint_resource_url, headers=header_parameters, params=query_parameters)
print(response.status_code)
print(response.text)


## Instantiate API objects

For testing, this cell only needs to be run once each time the notebook is opened. The instances can be reused for multiple API calls in later cells

In [ ]:
# Load the API key from plain text file
# Key is a single string on one line in the file with no trailing newline
api_key = load_credential('elsevier-api-key.txt', 'home')

# Initialize client
client = ElsClient(api_key)
# This sets an attribute for an institutional token that must be requested specially to
# access resources that are limited by the institutional subscriptions.
#client.inst_token = institutional_token



print('done')

## Author example

Doesn't work

In [ ]:
# Initialize author with uri
my_auth = ElsAuthor(
        uri = 'http://api.elsevier.com/content/author/AUTHOR_ID/7202909704')

In [ ]:
my_auth.read(client)

In [ ]:
my_auth.full_name


In [ ]:
## Author example
# Initialize author with uri
my_auth = ElsAuthor(
        uri = 'https://api.elsevier.com/content/author/author_id/7004367821')
# Read author data, then write to disk
if my_auth.read(client):
    print ("my_auth.full_name: ", my_auth.full_name)
    my_auth.write()
else:
    print ("Read author failed.")


In [ ]:
## Load list of documents from the API into affilation and author objects.
# Since a document list is retrieved for 25 entries at a time, this is
#  a potentially lenghty operation.

## Read all documents for example author, then write to disk
if my_auth.read_docs(client):
    print ("my_auth.doc_list has " + str(len(my_auth.doc_list)) + " items.")
    my_auth.write_docs()
else:
    print ("Read docs for author failed.")


## Affiliation example

Doesn't work

In [ ]:
## Affiliation example
# Initialize affiliation with ID as string
my_aff = ElsAffil(affil_id = '60101411')
if my_aff.read(client):
    print ("my_aff.name: ", my_aff.name)
    my_aff.write()
else:
    print ("Read affiliation failed.")


In [ ]:
## Load list of documents from the API into affilation and author objects.
# Since a document list is retrieved for 25 entries at a time, this is
#  a potentially lenghty operation.

## Read all documents for example affiliation, then write to disk
if my_aff.read_docs(client):
    print ("my_aff.doc_list has " + str(len(my_aff.doc_list)) + " items.")
    my_aff.write_docs()
else:
    print ("Read docs for affiliation failed.")


## Scopus abstract example

In [ ]:
## Scopus (Abtract) document example
# Initialize document with ID as integer
scp_doc = AbsDoc(scp_id = 84872135457)
if scp_doc.read(client):
    print ("scp_doc.title: ", scp_doc.title)
    print(json.dumps(scp_doc.data, indent=2))
    # The .write() method writes the results to a file in a "data" subdirectory of the
    # current working directory.
    #scp_doc.write()   
else:
    print ("Read document failed.")

## ScienceDirect full-text document example


In [ ]:
## ScienceDirect (full-text) document example using PII
doc = FullDoc(sd_pii = 'S1674927814000082')
#doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doc.read(client):
    print ("Document title: ", doc.title)
    print(json.dumps(doc.data, indent=2))
    #doc.write()   
else:
    print ("Read document failed.")


## Search using ElsSearch

Results in an authorization error: "The requestor is not authorized to access the requested view or fields of the resource"

In [ ]:
## Initialize search object and execute search
#srch = ElsSearch('authlast(keuskamp)','author')
aff_srch = ElsSearch('affil(amsterdam)','affiliation')
srch.execute(client)
print ("search has", len(srch.results), "results.")


Got results with the scopus search, but sciencedirect produces the same authorization error: "The requestor is not authorized to access the requested view or fields of the resource"

In [ ]:
## Initialize doc search object and execute search, retrieving all results
doc_srch = ElsSearch("AFFIL(vanderbilt) AND AUTHOR-NAME(baskauf s)",'scopus')
#doc_srch = ElsSearch("AFFIL(dartmouth) AND AUTHOR-NAME(lewis) AND PUBYEAR > 2011",'scopus')
#doc_srch = ElsSearch("star trek vs star wars",'sciencedirect')
#doc_srch.execute(client, get_all = True)
doc_srch.execute(client, get_all = False)
print ("doc_srch has", len(doc_srch.results), "results.")
print(json.dumps(doc_srch.results, indent=2))

# NOTE: for whatever reason, this automatically saved the results in a file called dump.json 
# in the active working directory. Not sure how this is controlled.